# Context creation using Wikipedia

We downloaded the 13GB Wikipedia Plaintext (2023-07-01) dataset from Kaggle. The wikipedia articles are stored in parquet files. We use only the wiki_2023_index.parquet file that contains the first sentences of the articles as context for the mdel. Then we use the Sentence Transformer library to embed the wikipedia articles and then used Faiss to create an index of the embeddings. We then used the index to retrieve the most similar wikipedia article for each question.

## Sources

* https://www.kaggle.com/datasets/jjinho/wikipedia-20230701/data?select=h.parquet

* https://github.com/facebookresearch/faiss/wiki

In [ ]:
!pip install kaggle
!pip install datasets
!pip install faiss-gpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=aea393b3d12c96fb4c2bc0dadc9cd781e900d7ca5aa61cc09813b70f7088e12e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from google.colab import files

# Upload kaggle.json file to google drive
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# Create kaggle directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import kaggle

# Specify the Kaggle dataset we want to download
dataset_name = 'jjinho/wikipedia-20230701'

# Download the specific file and unzip it
kaggle.api.authenticate()
kaggle.api.dataset_download_files(dataset_name, path='.', unzip=True)

In [ ]:
# Importing the libraries
import os
import pandas as pd
from datasets import load_dataset
import faiss
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
#IMportant parameters describing the code
SIM_MODEL = 'all-MiniLM-L6-v2'
DEVICE = 0
BATCH_SIZE = 8

In [ ]:
#Loading the questions
qna_df = pd.read_csv("https://raw.githubusercontent.com/csabi0312/DeepLProject/main/train.csv",index_col=0)

qna_df.head()

,prompt,A,B,C,D,E,answer
id,,,,,,,
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


Combine all wikipedia parquet files into one

In [ ]:
# Load Parquet files into a Hugging Face dataset
# Source: https://www.kaggle.com/datasets/jjinho/wikipedia-20230701/data?select=wiki_2023_index.parquet
wiki_dataset = load_dataset('parquet', data_files={'train': 'wiki_2023_index.parquet'}) # 1.76GB file

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Load pre-trained sentence transformer model
model = SentenceTransformer(SIM_MODEL)

# Encode the context sentences using the SentenceTransformer model
context_embeddings = model.encode(wiki_dataset['train']['context'],
                                  batch_size=BATCH_SIZE,
                                  device=DEVICE,
                                  show_progress_bar=True,
                                  convert_to_tensor=True,
                                  normalize_embeddings=True).half() # Use mixed-precision training (FP16) to reduce the memory footprint

# Convert the embeddings to a numpy array
context_embeddings_np = torch.stack(context_embeddings).cpu().numpy()

# Free up memory
del wiki_dataset
del context_embeddings

# Create a Faiss index and train it on the context embeddings
index = faiss.IndexFlatIP(context_embeddings_np.shape[1])
index.add(context_embeddings_np)

# Free up memory
del context_embeddings_np

# Function to retrieve most similar documents
def retrieve_most_similar(query, k=20):
    query_embedding = model.encode(query, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True)
    query_embedding = query_embedding.reshape(1, -1)  # Reshape for Faiss
    _, idx = index.search(query_embedding, k)
    return idx[0]

# Example usage
query_text = qna_df['prompt'][0]
print(f'example prompt {query_text}')
similar_documents_indices = retrieve_most_similar(query_text)

# Print similar documents
for idx in similar_documents_indices:
    print(wiki_dataset['train'][int(idx)]['title'])

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

RuntimeError: ignored

In [ ]:
# Create the context column from the wikipedia article
# Create an empty list to store the context for each prompt
context_list = []

# Loop through each prompt in the qna_df dataframe
for i in range(len(qna_df)):
    query_text = qna_df['prompt'][i]
    similar_documents_indices = retrieve_most_similar(query_text)

    # Get the first answer from the corresponding wiki_dataset
    context = wiki_dataset['train'][int(similar_documents_indices[0])]['context']


    context_list.append(context)

# Add the context_list as a new column "context" to the qna_df DataFrame
qna_df['context'] = context_list

# Save the Q&A DataFrame to a CSV file
qna_df.to_csv('openbook-qna-data.csv', index=False)

# Display the modified DataFrame
qna_df.head()

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
!cp openbook-qna-data.csv "drive/My Drive/"